In [1]:
import mysql.connector
from dotenv import load_dotenv
from decouple import config 
import os

In [28]:
load_dotenv()

connection = mysql.connector.connect(
    host= os.getenv("host"),
    user= os.getenv("user"),
    password= os.getenv("password"),
    database="music_trends"
)

cursor = connection.cursor(buffered=True)

In [3]:
music_trends = """CREATE DATABASE "music_trends";"""

In [ ]:
artist= """CREATE TABLE artists (
    artist_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255) UNIQUE NOT NULL
);"""
cursor.execute(artist) 

In [ ]:
songs="""CREATE TABLE songs (
    song_id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL
);""" 
cursor.execute(songs)

In [13]:
charts = """ CREATE TABLE charts (
    chart_id INT PRIMARY KEY AUTO_INCREMENT,
    song_id INT NOT NULL,
    artist_id INT NOT NULL,
    year INT NOT NULL,
    appearances INT,
    FOREIGN KEY (song_id) REFERENCES songs(song_id) ON DELETE CASCADE,
    FOREIGN KEY (artist_id) REFERENCES artists(artist_id) ON DELETE CASCADE
);
"""
cursor.execute(charts)

In [14]:
genres = """CREATE TABLE genres (
    genre_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) UNIQUE NOT NULL
);
"""
cursor.execute(genres)

In [15]:
genrestrends = """CREATE TABLE genretrends (
    trend_id INT PRIMARY KEY AUTO_INCREMENT,
    genre_id INT NOT NULL,
    year INT NOT NULL,
    appearances INT,
    FOREIGN KEY (genre_id) REFERENCES genres(genre_id) ON DELETE CASCADE
);
"""
cursor.execute(genrestrends)

In [ ]:
import pandas as pd


generos_df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\cleans\\Tendencia_generos.csv")


for genre in generos_df['Genre'].unique():
    cursor.execute("INSERT IGNORE INTO genres (name) VALUES (%s)", (genre,))

connection.commit()


cursor.execute("SELECT genre_id, name FROM genres")
genre_map = {name: gid for gid, name in cursor.fetchall()}


for row in generos_df.itertuples(index=False):
    genre_id = genre_map.get(row.Genre)
    if genre_id:
        cursor.execute(
            "INSERT INTO genretrends (genre_id, year, appearances) VALUES (%s, %s, %s)",
            (genre_id, row.Year, row.Frecuency)
        )
connection.commit()


canciones_df = pd.read_csv("C:\\Users\\Rober\\Desktop\\PROYECTS\\Music Trends Proyect\\CSV\\cleans\\Tendencia_canciones_artistas.csv")


for song in canciones_df['Song'].unique():
    cursor.execute("INSERT IGNORE INTO songs (title) VALUES (%s)", (song,))

connection.commit()

for artist in canciones_df['Artist'].unique():
    cursor.execute("INSERT IGNORE INTO artists (name) VALUES (%s)", (artist,))

connection.commit()

cursor.execute("SELECT song_id, title FROM songs")
song_map = {title: sid for sid, title in cursor.fetchall()}

cursor.execute("SELECT artist_id, name FROM artists")
artist_map = {name: aid for aid, name in cursor.fetchall()}



from datetime import datetime


for row in canciones_df.itertuples(index=False):
    song_id = song_map.get(row.Song)
    artist_id = artist_map.get(row.Artist)  

    try:
        year = datetime.strptime(row.Date, "%Y-%m-%d").year
    except Exception as e:
        print(f"Error con la fecha: {row.Date} - {e}")
        continue

    if song_id and artist_id:
        cursor.execute(
            "INSERT INTO charts (song_id, artist_id, year, appearances) VALUES (%s, %s, %s, %s)",
            (song_id, artist_id,year, row.Position)
        )



connection.commit()
cursor.close()
connection.close()
